### Questions about the text

* According to Chapter 1, what are some things that we humans might want technology to help us get?
We want to be heard.
We want to satisfy our curiosity.
We want it easy.
We want it now.

* What's the maximum number of characters in a Tweet?
140

* How many monthly active users does the book say that we can find on Twitter?
100 million

* What is the key difference between connections on Twitter and connections on Facebook?
Twitter you can follow any user, not mutual acceptance.

* Take a look at the tweet meta-data on page 22-25. Which fields do you think are most interesting?
retweet_count, favourites, time_created and location combined are interesting. We can see when the best time to post a tweet is.
User_mentions, location are interesting because we can see where a celebrities fanbase is for example.


# Exercises

In [1]:
import twitter
import json

CONSUMER_KEY = 'aF03g35xEjOm5OQuFw8PSuwLF'
CONSUMER_SECRET ='5Ifz6t0UPa6BxaaVm7eTQF4cOe3XcpVsX535nFkaLqy5bp7ehg'
OAUTH_TOKEN = '594338908-f0JBTAwwVGoZErJ4f1YNAgkj2gws7FQaFur4r2gs'
OAUTH_TOKEN_SECRET = 'EOR9ptKEtAc0LyhsEOAxsyEshWdgTcJKzXS6iBln2iB6h'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(domain='api.twitter.com', 
                              api_version='1.1',
                              auth=auth
                             )

In [3]:
WORLD_WOE_ID = 1
US_WOE_ID = 23424977
DEN_WOE_ID = 23424796

# Prefix ID with the underscore for query string parameterization.
# Without the underscore, the twitter package appends the ID value
# to the URL itself as a special case keyword argument.

# see https://dev.twitter.com/rest/reference/get/trends/place

# hitting the trends/place endpoint and the query specified is the location
den_trends = twitter_api.trends.place(_id=DEN_WOE_ID)
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter_api.trends.place(_id=US_WOE_ID)

In [31]:
# fancy print den_trends
print type(den_trends) # den_trends is a special twitter api class
output = json.dumps(den_trends)
# convert den_trends to a string
print type(output)
# parse the string and return as a python data structure
result = json.loads(output)
print type(result)
result

<class 'twitter.api.TwitterListResponse'>
<type 'str'>
<type 'list'>


[{u'as_of': u'2016-08-31T11:39:57Z',
  u'created_at': u'2016-08-31T11:33:30Z',
  u'locations': [{u'name': u'Denmark', u'woeid': 23424796}],
  u'trends': [{u'name': u'#transferdk',
    u'promoted_content': None,
    u'query': u'%23transferdk',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23transferdk'},
   {u'name': u'#DeadlineDay',
    u'promoted_content': None,
    u'query': u'%23DeadlineDay',
    u'tweet_volume': 102303,
    u'url': u'http://twitter.com/search?q=%23DeadlineDay'},
   {u'name': u'#tyve25',
    u'promoted_content': None,
    u'query': u'%23tyve25',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23tyve25'},
   {u'name': u'#dkgreen',
    u'promoted_content': None,
    u'query': u'%23dkgreen',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23dkgreen'},
   {u'name': u'#BeliebersRoastJustinParty',
    u'promoted_content': None,
    u'query': u'%23BeliebersRoastJustinParty',
    u'tweet_volume': 143291,
  

In [39]:
# creating sets (unique, unordered lists) for world and US trends with a list comprehension
world_trends_set = set([trend['name'] 
                        for trend in world_trends[0]['trends']])

us_trends_set = set([trend['name'] 
                     for trend in us_trends[0]['trends']]) 

den_trends_set = set([trend['name']
                      for trend in den_trends[0]['trends']])

# check for the same trend names shared by each set
common_trends = world_trends_set.intersection(us_trends_set)
# intersection between all three sets
common_trends_den = common_trends.intersection(den_trends_set)

print common_trends
print common_trends_den

set([u'#SwitchToReos', u'Amazon Dash', u'#8millionADHM', u'#HistoriasAlegresEn4Palabras', u'#FlipkartAssured', u'#WednesdayWisdom', u'#DeadlineDay', u'#IFA2016', u'#DoTheDifficult', u'#PrincessDiana'])
set([u'#DeadlineDay'])


In [5]:
# Import unquote to prevent url encoding errors in next_results
from urllib import unquote

q = '#DeadlineDay' 

count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
# hits /search/tweets.json?q=%23DeadlineDay.......
search_results = twitter_api.search.tweets(q=q, count=count)

statuses = search_results['statuses']

# Iterate through 5 more batches of results by following the cursor
# Each tweet contains a search_metadata field which contains a query for a similar search
# we iterate through 5 other other queries from search_metadata and add 100 results to the statuses list
for _ in range(5):
    print "Length of statuses", len(statuses)
    try:
        next_results = search_results['search_metadata']['next_results']
        print search_results['search_metadata']
        print search_results
        print next_results
    except KeyError, e: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    
    # kwargs are optional parameters, here we are passing the query we grabbed from search_metadata
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
print json.dumps(statuses[0], indent=1)

Length of statuses 100
{u'count': 100, u'completed_in': 0.086, u'max_id_str': u'793786413674180608', u'since_id_str': u'0', u'next_results': u'?max_id=792689389243658239&q=%23DeadlineDay&count=100&include_entities=1', u'refresh_url': u'?since_id=793786413674180608&q=%23DeadlineDay&include_entities=1', u'since_id': 0, u'query': u'%23DeadlineDay', u'max_id': 793786413674180608}
{u'search_metadata': {u'count': 100, u'completed_in': 0.086, u'max_id_str': u'793786413674180608', u'since_id_str': u'0', u'next_results': u'?max_id=792689389243658239&q=%23DeadlineDay&count=100&include_entities=1', u'refresh_url': u'?since_id=793786413674180608&q=%23DeadlineDay&include_entities=1', u'since_id': 0, u'query': u'%23DeadlineDay', u'max_id': 793786413674180608}, u'statuses': [{u'contributors': None, u'truncated': False, u'text': u'RT @aliensarrived: Is this a sign? Are aliens watching us?\nhttps://t.co/Thpnqrl77L seeks answers!\n\n#UFO #Aliens #DeadlineDay #GGJ16 https:/\u2026', u'is_quote_status': Fa

### Q: Extract the actual tweet text, and retweet count for all of the tweets you downloaded in example 1.5. Which tweet is the most popular one?

In [40]:
# testing just one output
print statuses[0]['retweet_count']
print statuses[0]['text']

231
RT @DeadlineDayLive: Sunderland are set to sign Steven Naismith from Norwich for £4m. (Source: Daily Mirror) #DeadlineDay https://t.co/PIjO…


In [41]:
# initialize varaibles
max_retweet = 0
tweets = {}
text_list = []
retweet_count_list = []

# iterate through the statuses
for status in statuses:
    # grab retweet_count and text from current status
    retweet_count = status['retweet_count']
    text = status['text']
    # save text and retweet_count to lists for further use
    text_list.append(text)
    retweet_count_list.append(retweet_count)
    # if retweet count is greater then current max, save text and retweet_count
    if retweet_count > max_retweet:
        popular_tweet = text
        max_retweet = retweet_count
                              
print popular_tweet
print max_retweet

RT @TSBible: Never forget! #DeadlineDay https://t.co/PG95Fsk2NG
1832
